In [2]:
import os

In [3]:
%pwd

'd:\\project_practice\\chicken_disease_classification_Project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'd:\\project_practice\\chicken_disease_classification_Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

In [7]:
from cnn_Classifier.constants import *
from cnn_Classifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]